In [1]:
%matplotlib inline

In [2]:
#dependencies

import pandas as pd
import requests
import gmaps
from config import g_key

## Heatmaps

In [3]:
#import city and weather data to avoid API call the next day

city_data_df = pd.read_csv("weather_data/cities.csv")

city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Long          float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# set up google api key

gmaps.configure(api_key=g_key)

In [5]:
#heatmap of max temp
#build lat long and weights lists


locations = city_data_df[["Lat","Long"]]
max_temp = city_data_df["Max Temp"]

#build basemap
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

#build heatmap layer and add to fig
heat_layer = gmaps.heatmap_layer(locations, weights = [max(0,temp) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#heatmap of humidity
#build lat long and weights lists


locations = city_data_df[["Lat","Long"]]
humidity = city_data_df["Humidity"]

#build basemap
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

#build heatmap layer and add to fig
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#heatmap of cloudiness
#build lat long and weights lists
locations = city_data_df[["Lat","Long"]]
clouds = city_data_df["Cloudiness"]

#build basemap
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

#build heatmap layer and add to fig
heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#heatmap of wind speed
#build lat long and weights lists
locations = city_data_df[["Lat","Long"]]
winds = city_data_df["Wind Speed"]

#build basemap
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

#build heatmap layer and add to fig
heat_layer = gmaps.heatmap_layer(locations, weights = winds, dissipating=False, max_intensity=50, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Vacation App

In [9]:
# get user inputs

min_temp = float(input("What is the minimum temperature you would like for your trip?\n"))
max_temp = float(input("What is the maximum temperature you would like for your trip?\n"))

In [10]:
#filter to the requested temps
temperate_df = city_data_df[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)].dropna()

#start hotel list from that filtered df
hotel_df = temperate_df[["City", "Country", "Max Temp", "Lat", "Long"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Lat,Long,Hotel Name
1,Payakumbuh,ID,69.53,-0.2202,100.6308,
2,Rikitea,PF,74.71,-23.1203,-134.9692,
6,Srednekolymsk,RU,65.52,67.4500,153.6833,
9,Torbay,CA,74.17,47.6666,-52.7314,
12,Yellowknife,CA,72.12,62.4560,-114.3525,
41,Kushmurun,KZ,65.73,52.4503,64.6311,
44,Namibe,AO,65.52,-15.1961,12.1522,
47,Vila Franca Do Campo,PT,70.21,37.7167,-25.4333,
54,Seymchan,RU,72.72,62.8833,152.4333,
57,Los Llanos De Aridane,ES,69.55,28.6585,-17.9182,


In [11]:
# link to documentation for nearby search
# https://developers.google.com/maps/documentation/places/web-service/search-nearby

#setting up constants for api search
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}


In [12]:
#iterate through the hotel df

for index, row in hotel_df.iterrows():
    #grab lat and long
    lat = row["Lat"]
    long = row["Long"]

    #add those to parameters dictionary
    params["location"] = f"{lat},{long}"

    #make the api request
    hotels = requests.get(url, params = params).json()
    
    try:
        #just pick the first hotel from the results
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        no_hotel_city = hotel_df.loc[index, "City"]
        print(f"Hotel in {no_hotel_city} not found...skipping")

Hotel in Srednekolymsk not found...skipping
Hotel in Seymchan not found...skipping
Hotel in Poum not found...skipping
Hotel in Berdigestyakh not found...skipping
Hotel in Myaundzha not found...skipping
Hotel in Lagos not found...skipping
Hotel in Birao not found...skipping
Hotel in Gberia Fotombu not found...skipping


In [13]:
hotel_df.tail(50)

,City,Country,Max Temp,Lat,Long,Hotel Name
267,Cap Malheureux,MU,68.47,-19.9842,57.6142,Pereybere Hotel & Spa
282,Cascais,PT,70.47,38.6979,-9.4215,Hotel Vila Galé Cascais
288,Bay Roberts,CA,70.52,47.5999,-53.2648,Stonewall Inn
291,Karaton,ID,72.86,-6.3424,106.1045,Parkir Mobil Wisata CAS Water Park
302,Sao Joao Da Barra,BR,70.99,-21.6403,-41.0511,Pousada Porto De Canoas
305,Skovorodino,RU,65.75,53.9833,123.9167,Amur
306,Ginda,IN,68.76,30.6945,78.4932,Sunil Bisht
309,Goderich,CA,72.91,43.7501,-81.7165,Harmony Inn
310,Caravelas,BR,70.86,-17.7125,-39.2481,Pousada dos Navegantes
312,Ferme-Neuve,CA,70.18,46.7001,-75.4493,Pourvoirie Club Fontbrune Ltee


In [17]:
#make heatmap just from hotels df

#build lat long and weights lists
locations = hotel_df[["Lat","Long"]]
max_temp = hotel_df["Max Temp"]

#build basemap
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

#build heatmap layer and add to fig
heat_layer = gmaps.heatmap_layer(locations, weights = [max(0,temp) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

#build markers and add to fig
#documentation on adding markers below
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#markers-and-symbols
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</d1>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))